1. 라이브러리 및 데이터

In [0]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!pip install tensorflow
!pip install tensorflow-gpu

     |████████████████████████████████| 512kB 18.5MB/s 
     |████████████████████████████████| 3.8MB 54.0MB/s 
ERROR: tensorflow-gpu 2.1.0 has requirement tensorboard<2.2.0,>=2.1.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow-gpu 2.1.0 has requirement tensorflow-estimator<2.2.0,>=2.1.0rc0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-estimator-2.1.0
  Found existing installation: tensorboard 2.1.1
    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
  Using cached https://files.pythonhosted.org/p

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import os

In [0]:
print(tf.__version__)
print(tf.test.is_gpu_available())

1.15.0
True


In [0]:
# 재생산성을 위해 시드 고정
np.random.seed(7)
random.seed(7)
tf.random.set_random_seed(7)

In [0]:
train = pd.read_csv("/gdrive/My Drive/dataset/AIFrenz/train.csv", index_col = 'id')
test = pd.read_csv("/gdrive/My Drive/dataset/AIFrenz/test.csv", index_col = 'id')

In [0]:
train.head()

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,Y00,Y01,Y02,Y03,Y04,Y05,Y06,Y07,Y08,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,1009.8,0.0,0.0,82.4,71.3,0.0,101.1,0.0,56.1,0.3,0.0,91.0,0.0,1002.5,1009.8,1.1,2.3,0.5,1008.7,12.3,1001.6,69.1,8.2,10.7,1010.1,0.0,256.4,0.0,77.2,62.6,0.0,12.5,11.5,11.0,20.0,20.0,10.5,10.0,10.5,10.0,7.0,7.5,7.0,9.0,10.0,9.5,9.0,8.0,9.0,NaN
1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,1009.9,0.0,0.0,81.2,121.8,0.0,165.3,0.0,0.0,0.4,0.0,91.9,0.0,1002.5,1009.9,0.0,0.0,0.0,1008.7,12.1,1001.7,70.3,8.3,10.3,1010.1,0.0,215.4,0.0,77.3,63.5,0.0,12.5,11.0,11.0,20.0,20.0,10.0,9.5,10.0,9.5,6.5,7.5,7.0,8.5,10.0,9.5,9.0,7.5,9.0,NaN
2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,1010.1,0.0,0.0,86.1,232.0,0.0,144.0,0.0,0.0,0.6,0.0,93.3,0.0,1002.4,1009.8,0.0,0.0,0.0,1008.6,12.1,1001.6,71.5,8.0,9.7,1010.0,0.0,235.2,0.0,77.3,63.9,0.0,12.0,11.0,11.0,19.5,19.5,10.0,9.0,10.0,9.0,6.5,7.5,6.5,8.0,9.5,9.5,8.5,7.5,8.5,NaN
3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,1010.0,0.0,0.0,87.7,0.0,0.0,115.9,0.0,294.1,0.1,0.0,93.4,0.0,1002.4,1009.7,0.0,0.0,0.2,1008.7,12.1,1001.5,73.2,7.7,9.4,1010.1,0.0,214.0,0.0,77.5,64.5,0.0,12.0,11.0,11.0,19.5,19.5,9.5,9.0,9.5,9.0,6.0,7.0,6.0,8.0,9.5,9.0,8.5,7.5,8.5,NaN
4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,1010.0,0.0,0.0,88.9,0.0,0.0,124.7,0.0,0.0,0.0,0.0,94.1,0.0,1002.4,1009.7,0.0,0.0,0.0,1008.7,12.0,1001.5,74.3,7.4,9.4,1010.1,0.0,174.9,0.0,78.0,65.0,0.0,12.0,10.5,10.5,19.0,19.5,9.5,8.5,9.5,8.5,6.0,7.0,6.0,7.5,9.5,9.0,8.5,7.5,8.5,NaN


In [0]:
train.tail()

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,Y00,Y01,Y02,Y03,Y04,Y05,Y06,Y07,Y08,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4747,19.9,987.6,0.9,0.8,0.0,1006.9,987.7,21.7,1007.5,1007.4,2.5,23.88,95.3,333.1,0.0,274.9,0.0,275.5,0.9,0.0,89.7,10.5,1000.4,1007.5,0.0,0.0,1.2,1006.0,23.8,999.4,89.9,17.7,19.1,1007.5,22.16,218.6,0.0,82.3,58.6,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4748,19.9,987.6,0.5,0.7,0.0,1006.8,987.7,21.6,1007.5,1007.4,2.5,23.88,95.6,216.4,0.0,277.3,0.0,247.0,1.0,0.0,90.1,10.5,1000.3,1007.5,0.0,0.0,1.3,1006.0,23.6,999.4,91.3,17.7,19.2,1007.5,22.16,161.7,0.0,82.5,59.1,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4749,19.7,987.7,0.9,0.6,0.0,1006.9,987.6,21.4,1007.4,1007.5,2.5,23.88,95.9,127.8,0.0,256.1,0.0,299.1,0.6,0.0,90.8,10.5,1000.4,1007.6,0.0,0.0,0.2,1006.0,23.6,999.5,90.2,17.8,19.2,1007.5,22.16,254.2,0.0,83.0,58.9,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4750,19.4,987.7,0.9,0.8,0.0,1006.9,987.8,21.3,1007.6,1007.5,2.5,23.88,95.9,323.8,0.0,255.1,0.0,0.0,0.6,0.0,91.5,10.5,1000.4,1007.6,0.0,0.0,0.1,1006.0,23.3,999.5,90.1,17.7,19.3,1007.6,22.16,300.0,0.0,83.2,59.8,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4751,19.1,987.6,1.0,0.3,0.0,1006.8,987.8,21.2,1007.5,1007.4,2.5,23.88,96.2,0.0,0.0,237.2,0.0,17.7,0.0,0.0,93.0,10.5,1000.3,1007.6,0.0,0.0,0.1,1006.1,23.2,999.5,89.6,17.7,19.5,1007.7,22.16,157.5,0.0,84.0,59.5,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0


In [0]:
test.head()

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4752,18.7,987.4,1.5,1.3,0.0,1006.8,987.7,21.1,1007.5,1007.2,0.0,0.0,96.1,333.6,0.0,199.6,0.0,36.0,1.1,0.0,94.7,0.0,1000.3,1007.6,0.0,0.0,0.1,1006.0,23.2,999.5,91.4,17.5,19.2,1007.6,0.0,194.7,0.0,84.0,59.9,0.0
4753,18.8,987.4,1.1,1.2,0.0,1006.7,987.7,21.2,1007.5,1007.2,0.0,0.0,96.2,335.5,0.0,194.0,0.0,127.7,0.6,0.0,95.3,0.0,1000.2,1007.5,0.0,0.0,0.3,1005.9,23.1,999.4,91.6,17.4,19.1,1007.5,0.0,173.1,0.0,84.0,60.5,0.0
4754,19.0,987.4,1.3,1.2,0.0,1006.6,987.6,21.2,1007.4,1007.2,0.0,0.0,96.4,0.0,0.0,172.9,0.0,59.0,0.0,0.0,93.9,0.0,1000.1,1007.4,0.0,0.0,0.7,1005.9,23.1,999.3,92.0,17.2,19.0,1007.5,0.0,208.6,0.0,85.0,60.8,0.0
4755,18.7,987.4,2.3,0.8,0.0,1006.6,987.6,21.1,1007.4,1007.2,0.0,0.0,96.5,252.9,0.0,153.5,0.0,54.9,0.4,0.0,94.4,0.0,1000.1,1007.4,0.0,0.0,0.6,1005.8,22.8,999.3,93.0,17.2,18.8,1007.4,0.0,185.0,0.0,85.8,61.2,0.0
4756,18.4,987.4,1.1,0.7,0.0,1006.7,987.7,20.9,1007.5,1007.2,0.0,0.0,96.6,238.2,0.0,173.8,0.0,48.8,0.8,0.0,95.7,0.0,1000.2,1007.3,0.0,0.0,0.7,1005.9,22.8,999.2,92.3,17.4,18.9,1007.5,0.0,162.8,0.0,87.3,61.7,0.0


In [0]:
train.shape  # 33일 간 데이터
test.shape   # 80일 간 데이터 

(4752, 59)

(11520, 40)

2. 데이터 전처리

In [0]:
# 기상청 데이터만 추출
X_train = train.loc[:,'X00':'X39']

# standardization을 위해 평균과 표준편차 구하기
MEAN = X_train.mean()
STD = X_train.std()

# 표준편차가 0일 경우 대비하여 1e-07 추가 
X_train = (X_train - MEAN) / (STD + 1e-07)

In [0]:
X_train.head()

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,1009.8,0.0,0.0,82.4,71.3,0.0,101.1,0.0,56.1,0.3,0.0,91.0,0.0,1002.5,1009.8,1.1,2.3,0.5,1008.7,12.3,1001.6,69.1,8.2,10.7,1010.1,0.0,256.4,0.0,77.2,62.6,0.0
1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,1009.9,0.0,0.0,81.2,121.8,0.0,165.3,0.0,0.0,0.4,0.0,91.9,0.0,1002.5,1009.9,0.0,0.0,0.0,1008.7,12.1,1001.7,70.3,8.3,10.3,1010.1,0.0,215.4,0.0,77.3,63.5,0.0
2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,1010.1,0.0,0.0,86.1,232.0,0.0,144.0,0.0,0.0,0.6,0.0,93.3,0.0,1002.4,1009.8,0.0,0.0,0.0,1008.6,12.1,1001.6,71.5,8.0,9.7,1010.0,0.0,235.2,0.0,77.3,63.9,0.0
3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,1010.0,0.0,0.0,87.7,0.0,0.0,115.9,0.0,294.1,0.1,0.0,93.4,0.0,1002.4,1009.7,0.0,0.0,0.2,1008.7,12.1,1001.5,73.2,7.7,9.4,1010.1,0.0,214.0,0.0,77.5,64.5,0.0
4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,1010.0,0.0,0.0,88.9,0.0,0.0,124.7,0.0,0.0,0.0,0.0,94.1,0.0,1002.4,1009.7,0.0,0.0,0.0,1008.7,12.0,1001.5,74.3,7.4,9.4,1010.1,0.0,174.9,0.0,78.0,65.0,0.0


In [0]:
X_train.head()

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-2.236377,0.053243,-0.288997,-0.911345,-0.286742,0.284562,0.174782,-2.160752,0.315216,0.396435,-0.30283,-1.01241,0.636957,-0.911630,0.0,-0.749052,0.0,-0.911858,-0.932848,0.0,0.894216,-0.279076,0.215816,0.277681,0.105324,-1.044024,-0.701013,0.332246,-2.314454,0.232454,0.159919,-2.109779,-1.766623,0.297981,-1.015881,0.482017,-0.274946,0.577720,0.240703,-0.234576
1,-2.321347,0.083006,0.177482,0.477320,-0.286742,0.284562,0.174782,-2.183598,0.343622,0.424634,-0.30283,-1.01241,0.583093,-0.400432,0.0,-0.092515,0.0,-1.395715,-0.846270,0.0,0.935443,-0.279076,0.215816,0.306941,-0.902636,-1.061523,-1.103946,0.332246,-2.361789,0.261951,0.215666,-2.091664,-1.842501,0.297981,-1.015881,0.070290,-0.274946,0.582841,0.289700,-0.234576
2,-2.300105,0.112769,-0.382293,0.904602,-0.286742,0.255107,0.204162,-2.183598,0.372029,0.481033,-0.30283,-1.01241,0.803040,0.715094,0.0,-0.310338,0.0,-1.395715,-0.673113,0.0,0.999573,-0.279076,0.186122,0.277681,-0.902636,-1.061523,-1.103946,0.303230,-2.361789,0.232454,0.271413,-2.146009,-1.956318,0.268916,-1.015881,0.269124,-0.274946,0.582841,0.311477,-0.234576
3,-2.300105,0.083006,-0.009110,-0.804525,-0.286742,0.255107,0.174782,-2.206443,0.343622,0.452834,-0.30283,-1.01241,0.874860,-1.633382,0.0,-0.597701,0.0,1.140868,-1.106005,0.0,1.004154,-0.279076,0.186122,0.248421,-0.902636,-1.061523,-0.942773,0.332246,-2.361789,0.202958,0.350389,-2.200355,-2.013226,0.297981,-1.015881,0.056231,-0.274946,0.593083,0.344142,-0.234576
4,-2.342589,0.083006,-0.662181,0.263679,-0.286742,0.255107,0.204162,-2.206443,0.372029,0.452834,-0.30283,-1.01241,0.928724,-1.633382,0.0,-0.507708,0.0,-1.395715,-1.192583,0.0,1.036219,-0.279076,0.186122,0.248421,-0.902636,-1.061523,-1.103946,0.332246,-2.385457,0.202958,0.401490,-2.254700,-2.013226,0.297981,-1.015881,-0.336416,-0.274946,0.618688,0.371363,-0.234576


In [0]:
X_train.tail()

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,dummy
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4747,-0.069655,-0.303908,-0.568885,-0.697704,-0.286742,-0.422352,-0.383432,0.009577,-0.366541,-0.28035,0.449182,1.183747,1.216003,1.738504,0.0,1.028302,0.0,0.980445,-0.413378,0.0,0.834666,2.29506,-0.407749,-0.39530,-0.902636,-1.061523,-0.136906,-0.451164,0.407308,-0.416474,1.126205,-0.388845,-0.173188,-0.457686,0.930244,0.102425,-0.274946,0.838894,0.022936,-0.126289,0
4748,-0.069655,-0.303908,-0.942069,-0.804525,-0.286742,-0.451807,-0.383432,-0.013269,-0.366541,-0.28035,0.449182,1.183747,1.229469,0.557180,0.0,1.052845,0.0,0.734636,-0.326800,0.0,0.852989,2.29506,-0.437442,-0.39530,-0.902636,-1.061523,-0.056319,-0.451164,0.359973,-0.416474,1.191244,-0.388845,-0.154219,-0.457686,0.930244,-0.468972,-0.274946,0.849136,0.050157,-0.126289,0
4749,-0.112139,-0.274146,-0.568885,-0.911345,-0.286742,-0.422352,-0.412812,-0.058960,-0.394948,-0.25215,0.449182,1.183747,1.242935,-0.339695,0.0,0.836045,0.0,1.183993,-0.673113,0.0,0.885054,2.29506,-0.407749,-0.36604,-0.902636,-1.061523,-0.942773,-0.451164,0.359973,-0.386977,1.140142,-0.370730,-0.154219,-0.457686,0.930244,0.459924,-0.274946,0.874742,0.039269,-0.126289,0
4750,-0.175867,-0.274146,-0.568885,-0.697704,-0.286742,-0.422352,-0.354052,-0.081806,-0.338135,-0.25215,0.449182,1.183747,1.242935,1.644362,0.0,0.825819,0.0,-1.395715,-0.673113,0.0,0.917120,2.29506,-0.407749,-0.36604,-0.902636,-1.061523,-1.023359,-0.451164,0.288971,-0.386977,1.135497,-0.388845,-0.135249,-0.428622,0.930244,0.919853,-0.274946,0.884984,0.088266,-0.126289,0
4751,-0.239594,-0.303908,-0.475589,-1.231806,-0.286742,-0.451807,-0.354052,-0.104651,-0.366541,-0.28035,0.449182,1.183747,1.256401,-1.633382,0.0,0.642766,0.0,-1.243054,-1.192583,0.0,0.985831,2.29506,-0.437442,-0.36604,-0.902636,-1.061523,-1.023359,-0.422149,0.265303,-0.386977,1.112269,-0.388845,-0.097310,-0.399558,0.930244,-0.511149,-0.274946,0.925952,0.071934,-0.126289,0


In [0]:
# RNN 모델에 입력 할 수 있는 시계열 형태로 데이터 변환 
def convert_to_timeseries(df, interval):
    sequence_list = []
    target_list = []
    
    for i in tqdm(range(df.shape[0] - interval)):
        sequence_list.append(np.array(df.iloc[i:i+interval,:-1]))
        target_list.append(df.iloc[i+interval,-1])
    
    sequence = np.array(sequence_list)
    target = np.array(target_list)
    
    return sequence, target

In [0]:
# 토론 글에 기반하여, Y15와 Y16 지점의 데이터가 Y18과 상관관계가 높음. 그래서 두 지점의 데이터를 학습하기로 결정.
y_columns = ['Y15','Y16']

In [0]:
y_columns

['Y15', 'Y16']

In [0]:
# t시점 이전 120분의 데이터로 t시점의 온도를 추정할 수 있는 학습데이터 형성
sequence = np.empty((0, 12, 40))
target = np.empty((0,))
for column in y_columns :
    
    concat = pd.concat([X_train, train[column]], axis = 1)

    _sequence, _target = convert_to_timeseries(concat.head(6*24*30), interval = 12)

    sequence = np.vstack((sequence, _sequence))
    target = np.hstack((target, _target))


100%|██████████| 4308/4308 [00:01<00:00, 3326.91it/s]


In [0]:
# convert_to_timeseries 함수를 쓰기 위한 dummy feature 생성
X_train['dummy'] = 0

In [0]:
X_train.head()

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,dummy
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-2.236377,0.053243,-0.288997,-0.911345,-0.286742,0.284562,0.174782,-2.160752,0.315216,0.396435,-0.30283,-1.01241,0.636957,-0.911630,0.0,-0.749052,0.0,-0.911858,-0.932848,0.0,0.894216,-0.279076,0.215816,0.277681,0.105324,-1.044024,-0.701013,0.332246,-2.314454,0.232454,0.159919,-2.109779,-1.766623,0.297981,-1.015881,0.482017,-0.274946,0.577720,0.240703,-0.234576,0
1,-2.321347,0.083006,0.177482,0.477320,-0.286742,0.284562,0.174782,-2.183598,0.343622,0.424634,-0.30283,-1.01241,0.583093,-0.400432,0.0,-0.092515,0.0,-1.395715,-0.846270,0.0,0.935443,-0.279076,0.215816,0.306941,-0.902636,-1.061523,-1.103946,0.332246,-2.361789,0.261951,0.215666,-2.091664,-1.842501,0.297981,-1.015881,0.070290,-0.274946,0.582841,0.289700,-0.234576,0
2,-2.300105,0.112769,-0.382293,0.904602,-0.286742,0.255107,0.204162,-2.183598,0.372029,0.481033,-0.30283,-1.01241,0.803040,0.715094,0.0,-0.310338,0.0,-1.395715,-0.673113,0.0,0.999573,-0.279076,0.186122,0.277681,-0.902636,-1.061523,-1.103946,0.303230,-2.361789,0.232454,0.271413,-2.146009,-1.956318,0.268916,-1.015881,0.269124,-0.274946,0.582841,0.311477,-0.234576,0
3,-2.300105,0.083006,-0.009110,-0.804525,-0.286742,0.255107,0.174782,-2.206443,0.343622,0.452834,-0.30283,-1.01241,0.874860,-1.633382,0.0,-0.597701,0.0,1.140868,-1.106005,0.0,1.004154,-0.279076,0.186122,0.248421,-0.902636,-1.061523,-0.942773,0.332246,-2.361789,0.202958,0.350389,-2.200355,-2.013226,0.297981,-1.015881,0.056231,-0.274946,0.593083,0.344142,-0.234576,0
4,-2.342589,0.083006,-0.662181,0.263679,-0.286742,0.255107,0.204162,-2.206443,0.372029,0.452834,-0.30283,-1.01241,0.928724,-1.633382,0.0,-0.507708,0.0,-1.395715,-1.192583,0.0,1.036219,-0.279076,0.186122,0.248421,-0.902636,-1.061523,-1.103946,0.332246,-2.385457,0.202958,0.401490,-2.254700,-2.013226,0.297981,-1.015881,-0.336416,-0.274946,0.618688,0.371363,-0.234576,0


In [0]:
# train set에서 도출된 평균과 표준편차로 standardization 실시 
test = (test - MEAN) / (STD + 1e-07)

In [0]:
# convert_to_timeseries 함수를 쓰기 위한 dummy feature 생성
test['dummy'] = 0

In [0]:
X_train.shape

(4752, 41)

In [0]:
# train과 test 기간을 합쳐서 120분 간격으로 학습데이터 재구축
X_test, _ = convert_to_timeseries(pd.concat([X_train, test], axis = 0), interval=12)

100%|██████████| 16260/16260 [00:06<00:00, 2476.11it/s]


In [0]:
X_test.shape

(16260, 12, 40)

In [0]:
# test set 기간인 후반부 80일에 맞게 자르기 
X_test = X_test[-11520:, :, :]   ## 뒤에서부터 11520개

In [0]:
X_test.shape

(11520, 12, 40)

In [0]:
# 만들어 두었던 dummy feature 제거
X_train.drop('dummy', axis = 1, inplace = True)
test.drop('dummy', axis = 1, inplace = True)

3. 변수 선택 및 모델 구축

In [0]:
# 간단한 lstm 모델 구축하기 
simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(128, input_shape=sequence.shape[-2:]),  ## input_shape : (12, 40)
    tf.keras.layers.Dense(256, activation='linear'),
    tf.keras.layers.Dense(128, activation='linear'),
    tf.keras.layers.Dense(1)
])

simple_lstm_model.compile(optimizer='adam', loss='mse')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


4. 모델 학습 및 검증

In [0]:
# loss가 4미만으로 떨어지면 학습 종료 시키는 기능
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = None):
        if(logs.get('loss') < 4):
            print('\n Loss is under 4, cancelling training')
            self.model.stop_training = True

In [0]:
callbacks = myCallback()

In [0]:
# 모델 학습
simple_lstm_model.fit(    
    sequence, target,
    epochs=60,
    batch_size=128,
    verbose=2,
    shuffle=False,
    callbacks = [callbacks]
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 8616 samples
Epoch 1/60
8616/8616 - 9s - loss: 163.0117
Epoch 2/60
8616/8616 - 1s - loss: 12.8587
Epoch 3/60
8616/8616 - 1s - loss: 7.0696
Epoch 4/60
8616/8616 - 1s - loss: 5.6218
Epoch 5/60
8616/8616 - 1s - loss: 5.1560
Epoch 6/60
8616/8616 - 1s - loss: 5.1739
Epoch 7/60
8616/8616 - 1s - loss: 5.4304
Epoch 8/60
8616/8616 - 1s - loss: 6.3075
Epoch 9/60
8616/8616 - 1s - loss: 7.1049
Epoch 10/60
8616/8616 - 1s - loss: 8.2125
Epoch 11/60
8616/8616 - 1s - loss: 7.3714
Epoch 12/60
8616/8616 - 1s - loss: 5.6903
Epoch 13/60
8616/8616 - 1s - loss: 5.5199
Epoch 14/60
8616/8616 - 1s - loss: 4.8865
Epoch 15/60
8616/8616 - 1s - loss: 4.7450
Epoch 16/60
8616/8616 - 1s - loss: 6.1137
Epoch 17/60
8616/8616 - 1s - loss: 6.3139
Epoch 18/60
8616/8616 - 1s - loss: 4.6660
Epoch 19/60
8616/8616 - 1s - loss: 4.2610
Epoch 20/60
8616/8616 - 1s - loss: 4.5748
Epoch 21/60
8616/8616 - 1s - loss: 5.4268
Epoch 22

In [0]:
# LSTM 레이어는 고정
simple_lstm_model.layers[0].trainable = False

In [0]:
# fine tuning 할 때 사용할 학습데이터 생성 (Y18)
finetune_X, finetune_y = convert_to_timeseries(pd.concat([X_train.tail(432), train['Y18'].tail(432)], axis = 1), interval=12)

100%|██████████| 420/420 [00:00<00:00, 2001.07it/s]


In [0]:
finetune_y.shape

(420,)

In [0]:
qq

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,Y18
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4320,-0.197109,-0.274146,-0.568885,0.797781,-0.286742,-0.186714,-0.295293,-0.196033,-0.281322,-0.280350,-0.302830,-1.012410,1.121740,0.011564,0.0,1.415884,0.0,-1.395715,-0.759692,0.0,0.710985,-0.279076,-0.199894,-0.190480,0.105324,0.897575,-1.103946,-0.248058,-0.255382,-0.209997,1.019356,-0.334500,-0.249066,-0.225173,-1.015881,1.457107,-0.274946,0.925952,1.057328,-0.234576,20.5
4321,-0.260836,-0.303908,0.364074,0.797781,-0.286742,-0.186714,-0.295293,-0.241725,-0.281322,-0.280350,-0.302830,-1.012410,1.144183,1.579577,0.0,1.376001,0.0,-1.395715,-1.106005,0.0,0.811762,-0.279076,-0.199894,-0.190480,-0.536105,0.935616,-1.023359,-0.248058,-0.326384,-0.209997,0.963609,-0.388845,-0.192158,-0.225173,-1.015881,0.950984,-0.274946,0.946436,1.144434,-0.234576,20.5
4322,-0.239594,-0.303908,0.457370,-0.056782,-0.286742,-0.157259,-0.265913,-0.264570,-0.252915,-0.280350,-0.302830,-1.012410,1.139694,-1.633382,0.0,1.224650,0.0,-1.395715,-1.192583,0.0,0.811762,-0.279076,-0.170201,-0.190480,-0.811003,1.551876,-1.103946,-0.248058,-0.326384,-0.209997,0.931090,-0.443191,-0.154219,-0.225173,-1.015881,0.993161,-0.274946,0.972042,1.182543,-0.234576,20.5
4323,-0.218351,-0.274146,0.270778,0.050038,-0.286742,-0.127805,-0.265913,-0.264570,-0.252915,-0.252150,-0.302830,-1.012410,1.144183,-1.633382,0.0,1.260442,0.0,-1.395715,-1.192583,0.0,0.839247,-0.279076,-0.140507,-0.190480,-0.902636,-1.061523,-1.103946,-0.219042,-0.350052,-0.209997,0.903216,-0.425076,-0.135249,-0.196109,-1.015881,1.076510,-0.274946,0.972042,1.226097,-0.234576,20.5
4324,-0.218351,-0.244383,-0.102406,-0.056782,-0.286742,-0.127805,-0.265913,-0.264570,-0.281322,-0.223951,-0.302830,-1.012410,1.153161,-1.633382,0.0,1.053868,0.0,-1.395715,-1.192583,0.0,0.834666,-0.279076,-0.140507,-0.161219,-0.902636,-1.061523,-1.103946,-0.219042,-0.373719,-0.180500,0.893925,-0.443191,-0.097310,-0.196109,-1.015881,1.132746,-0.274946,0.972042,1.236985,-0.234576,20.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4747,-0.069655,-0.303908,-0.568885,-0.697704,-0.286742,-0.422352,-0.383432,0.009577,-0.366541,-0.280350,0.449182,1.183747,1.216003,1.738504,0.0,1.028302,0.0,0.980445,-0.413378,0.0,0.834666,2.295060,-0.407749,-0.395300,-0.902636,-1.061523,-0.136906,-0.451164,0.407308,-0.416474,1.126205,-0.388845,-0.173188,-0.457686,0.930244,0.102425,-0.274946,0.838894,0.022936,-0.126289,21.5
4748,-0.069655,-0.303908,-0.942069,-0.804525,-0.286742,-0.451807,-0.383432,-0.013269,-0.366541,-0.280350,0.449182,1.183747,1.229469,0.557180,0.0,1.052845,0.0,0.734636,-0.326800,0.0,0.852989,2.295060,-0.437442,-0.395300,-0.902636,-1.061523,-0.056319,-0.451164,0.359973,-0.416474,1.191244,-0.388845,-0.154219,-0.457686,0.930244,-0.468972,-0.274946,0.849136,0.050157,-0.126289,21.5
4749,-0.112139,-0.274146,-0.568885,-0.911345,-0.286742,-0.422352,-0.412812,-0.058960,-0.394948,-0.252150,0.449182,1.183747,1.242935,-0.339695,0.0,0.836045,0.0,1.183993,-0.673113,0.0,0.885054,2.295060,-0.407749,-0.366040,-0.902636,-1.061523,-0.942773,-0.451164,0.359973,-0.386977,1.140142,-0.370730,-0.154219,-0.457686,0.930244,0.459924,-0.274946,0.874742,0.039269,-0.126289,21.5


In [0]:
finetune_X.shape
finetune_y.shape

(420, 12, 40)

(420,)

In [0]:
# LSTM 레이어는 고정 시켜두고, DNN 레이어에 대해서 fine tuning 진행 (Transfer Learning)
finetune_history = simple_lstm_model.fit(
            finetune_X, finetune_y,
            epochs=20,
            batch_size=64,
            shuffle=False,
            verbose = 2)

Train on 420 samples
Epoch 1/20
420/420 - 0s - loss: 0.5071
Epoch 2/20
420/420 - 0s - loss: 0.5109
Epoch 3/20
420/420 - 0s - loss: 0.4766
Epoch 4/20
420/420 - 0s - loss: 0.4694
Epoch 5/20
420/420 - 0s - loss: 0.4515
Epoch 6/20
420/420 - 0s - loss: 0.4388
Epoch 7/20
420/420 - 0s - loss: 0.4292
Epoch 8/20
420/420 - 0s - loss: 0.4149
Epoch 9/20
420/420 - 0s - loss: 0.4068
Epoch 10/20
420/420 - 0s - loss: 0.3944
Epoch 11/20
420/420 - 0s - loss: 0.3852
Epoch 12/20
420/420 - 0s - loss: 0.3761
Epoch 13/20
420/420 - 0s - loss: 0.3659
Epoch 14/20
420/420 - 0s - loss: 0.3587
Epoch 15/20
420/420 - 0s - loss: 0.3485
Epoch 16/20
420/420 - 0s - loss: 0.3412
Epoch 17/20
420/420 - 0s - loss: 0.3325
Epoch 18/20
420/420 - 0s - loss: 0.3247
Epoch 19/20
420/420 - 0s - loss: 0.3184
Epoch 20/20
420/420 - 0s - loss: 0.3110


In [0]:
# 예측하기 
finetune_pred = simple_lstm_model.predict(X_test)

In [0]:
finetune_pred

array([[21.351072],
       [20.983845],
       [20.703867],
       ...,
       [25.598726],
       [25.265223],
       [24.77508 ]], dtype=float32)

In [0]:
# 제출 파일 만들기
submit = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':finetune_pred.reshape(1,-1)[0]})

In [0]:
submit.shape

(11520, 2)

In [0]:
submit.head()

,id,Y18
0,4752,21.351072
1,4753,20.983845
2,4754,20.703867
3,4755,20.250223
4,4756,19.936071


In [0]:
submit.tail()

,id,Y18
11515,16267,25.124069
11516,16268,25.398155
11517,16269,25.598726
11518,16270,25.265223
11519,16271,24.775080


In [0]:
submit.to_csv('/gdrive/My Drive/dataset/AIFrenz/baseline_result.csv', index = False)